# NYC Taxi Rides

In this notebook we show some more advanced features that are useful for building composed views, a.k.a., dashboards.

In [1]:
HOST = 'localhost'
PORT = 8003

# If using Binder, set this to the current host: https://hub.{domain}/user-{username}-{hash}
LOCATION = ''

if LOCATION:
    # Use the jupyterhub proxy address to reach our server
    API_URL = LOCATION.replace('/lab', '') + f'/proxy/{PORT}/api/v1'
else:
    API_URL = f'http://{HOST}:{PORT}/api/v1'

%load_ext autoreload
%autoreload 2

### Download the data

In [2]:
import pandas as pd
from utils import download_file

nyc_data = pd.read_csv('data/nyc-taxi.csv')

for data in nyc_data.iterrows():
    download_file(*data[1])

File already exist. To overwrite pass `overwrite=True`
File already exist. To overwrite pass `overwrite=True`


### Create tilesets

In [3]:
from higlass.tilesets import cooler

pickup = cooler('data/nyctaxi-pickup.count.1m.mcool', name='Total Pickups')
dropoff = cooler('data/nyctaxi-dropoff.count.1m.mcool', name='Total Dropoffs')

### Visualize pickups and drop-offs

In [4]:
from higlass import Track, View, display

nyc_x=[11789206, 11827005]
nyc_y=[15044734, 15081957]

In [10]:
view_config = {
    'width': 6,
    'initialXDomain': nyc_x,
    'initialYDomain': nyc_y,
}

pickups_view = View(
    x=0,
    tracks=[Track('heatmap', tileset=pickup, height=400, server=API_URL)],
    **view_config
)

dropoffs_view = View(
    x=6,
    tracks=[Track('heatmap', tileset=dropoff, height=400, server=API_URL)],
    **view_config
)

widget, server, viewconf = display([pickups_view, dropoffs_view], 
                                   host=HOST, server_port=PORT)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'eNSVGiFbRtup2…

### Synchronize the viewport location and zoom level

In [8]:
widget, server, viewconf = display(
    views=[pickups_view, dropoffs_view],
    location_syncs=[(pickups_view, dropoffs_view)],
    zoom_syncs=[(pickups_view, dropoffs_view)],
    host=HOST,
    server_port=PORT
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'Gyjlm90OQOm6K…

### Add crosshairs and tooltips

In [14]:
## Important change ##########
track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'showMousePosition': True,
        'mousePositionColor': 'black',
        'showTooltip': True,
    },
    'server': API_URL,
}
##############################

pickups_view = View(
    x=0,
    tracks=[Track(tileset=pickup, **track_config)],
    **view_config
)

dropoffs_view = View(
    x=6,
    tracks=[Track(tileset=dropoff, **track_config)],
    **view_config
)

widget, server, viewconf = display(
    views=[pickups_view, dropoffs_view],
    location_syncs=[(pickups_view, dropoffs_view)],
    zoom_syncs=[(pickups_view, dropoffs_view)],
    host=HOST,
    server_port=PORT
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'PM90XNiGQBiLn…

### Overview & Details with Viewport projections

In [15]:
from copy import deepcopy

## Important change ##########
view_config_details = deepcopy(view_config)
view_config_details['y'] = 6
view_config_details['initialXDomain'] = [11811324, 11817197]
view_config_details['initialYDomain'] = [15057338, 15060882]
##############################

new_track_config = deepcopy(track_config)
new_track_config['height'] = 300

pickups_view_details = View(
    x=0,
    tracks=[Track(tileset=pickup, **new_track_config)],
    **view_config_details
)

dropoffs_view_details = View(
    x=6,
    tracks=[Track(tileset=dropoff, **new_track_config)],
    **view_config_details
)

pickups_view_overview = View(
    x=0,
    tracks=[
        Track(tileset=pickup, **new_track_config),
        ## Important change ##########
        Track(
            'viewport-projection-center',
            fromViewUid=pickups_view_details.uid,
            server=API_URL
        )
        ##############################
    ],
    **view_config
)

dropoffs_view_overview = View(
    x=6,
    tracks=[
        Track(tileset=dropoff, **new_track_config),
        ## Important change ##########
        Track(
            'viewport-projection-center',
            fromViewUid=dropoffs_view_details.uid,
            server=API_URL
        )
        ##############################
    ],
    **view_config
)

widget, server, viewconf = display(
    views=[
        pickups_view_overview, dropoffs_view_overview,
        pickups_view_details, dropoffs_view_details
    ],
    location_syncs=[
        (pickups_view_overview, dropoffs_view_overview),
        (pickups_view_details, dropoffs_view_details)
    ],
    zoom_syncs=[
        (pickups_view_overview, dropoffs_view_overview),
        (pickups_view_details, dropoffs_view_details)
    ],
    host=HOST,
    server_port=PORT
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'IoOQ2-egSgaNu…

### Compare different attributes through data operations

In [16]:
pickups_track = Track(tileset=pickup, **track_config)
dropoffs_track = Track(tileset=dropoff, **track_config)

pickups_view = View(
    x=0, tracks=[pickups_track], **view_config
)

dropoffs_view = View(
    x=6, tracks=[dropoffs_track], **view_config
)

widget, server, viewconf = display(
    views=[pickups_view, dropoffs_view],
    location_syncs=[(pickups_view, dropoffs_view)],
    zoom_syncs=[(pickups_view, dropoffs_view)],
    host=HOST,
    server_port=PORT
)

widget

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'U9Th-x0FRrip8…

Let's divide pickups by dropoff by directly adjusting the view config

In [17]:
from higlass import ViewConf
from higlass.viewer import HiGlassDisplay

diverging_colormap = [
    '#0085cc',
    '#62cef6',
    '#eeeeee',
    '#ff66b8',
    '#bf0066',
]

diff_track = (
    pickups_track
        .change_options(
            name='Pickups divided by dropoffs',
            colorRange=diverging_colormap,
            valueScaleMin=0.02,
            valueScaleMax=50)
        .change_attributes(
            server=server.api_address,
            ## Important change ##########
            data={
                'type': 'divided',
                'children': [{
                    'server': API_URL,
                    'tilesetUid': pickup.uuid
                }, {
                    'server': API_URL,
                    'tilesetUid': dropoff.uuid
                }]
            }
            ##############################
        )
)

diff_view_config = deepcopy(view_config)
diff_view_config['width'] = 12
diff_view = View(tracks=[diff_track], **diff_view_config)

# Finally, we need to create a new view config and pass that into the widget
HiGlassDisplay(viewconf=ViewConf([diff_view]).to_dict())

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'H1fTUWtJTU64hHHasWFEBA', 'tracks': {'top': [], '…

- **Pink** indicates areas with more *pickups*
- **Blue** indicates areas with more *dropoffs*

### Superimpose tracks

In [18]:
## Important change ##########
diverging_colormap = [
    'rgba(0, 133, 204, 0.66)',
    'rgba(98, 206, 246, 0.33)',
    'rgba(238, 238, 238, 0.01)',
    'rgba(255, 102, 184, 0.33)',
    'rgba(191, 0, 102, 0.66)',
]

new_diff_track = diff_track.change_options(
    # This is critical
    backgroundColor='transparent',
    colorRange=diverging_colormap
)
##############################

mapbox_track_config = {
    'track_type': 'mapbox',
    'position': 'center',
    'height': 400,
    'options': {
        'accessToken': 'pk.eyJ1IjoiZmxla3NjaGFzIiwiYSI6ImNqeHRzMWJvZTB4dmQzZ3Q1cThqejB2dGsifQ.j_01IEuxiF8-JhX1BGKueA',
        'style': 'light-v9',
        'minPos': [1],
        'maxPos': [40075016],
        'name': '© Mapbox',
        'labelPosition': 'bottomLeft',
    }
}

new_diff_view = View(
    tracks=[Track(**mapbox_track_config), new_diff_track],
    **diff_view_config
)

HiGlassDisplay(viewconf=ViewConf([new_diff_view]).to_dict())

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'KYy_W6oFQNmQU5XRWNjtpA', 'tracks': {'top': [], '…

### Finally... let's turn the lights off

Everyone loves dark mode! We love it too 😍

In [19]:
heatmap_track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'backgroundColor': 'transparent',
        ## Important change ##########
        'colorRange': [
            'rgba(0, 0, 0, 0.01)',
            'rgba(34, 46, 84, 0.25)',
            'rgba(68, 141, 178, 0.5)',
            'rgba(104, 191, 48, 0.7)',
            'rgba(253, 255, 84, 0.8)',
            'rgba(253, 255, 255, 0.9)',
        ],
        ##############################
        'colorbarBackgroundColor': 'black',
        'colorbarBackgroundOpacity': 0.5,
        'labelColor': '#ffffff',
        'labelTextOpacity': 0.66,
        'labelBackgroundColor': 'black',
        'labelBackgroundOpacity': 0.5,
    },
    'server': API_URL,
}
mapbox_track_config = {
    'track_type': 'mapbox',
    'position': 'center',
    'height': 400,
    'options': {
        'accessToken': 'pk.eyJ1IjoiZmxla3NjaGFzIiwiYSI6ImNqZXB2aWd4NDBmZTIzM3BjdGZudTFob2oifQ.Jnmp1xWJyS4_lRhzrZAFBQ',
        'style': 'dark-v8',
        'minPos': [1],
        'maxPos': [40075016],
        'name': '© Mapbox',
        'labelPosition': 'bottomLeft',
        'labelColor': '#ffffff',
        'labelTextOpacity': 0.66,
        'labelBackgroundColor': 'black',
        'labelBackgroundOpacity': 0.5,
    },
    'server': API_URL,
}

pickups_in_dark = View(
    tracks=[
        Track(**mapbox_track_config),
        Track(tileset=dropoff, **heatmap_track_config),
    ],
    **diff_view_config
)

## Important change ##########
widget, server, viewconf = display([pickups_in_dark], dark_mode=True, 
                                   host=HOST, server_port=PORT)
##############################

widget

HiGlassDisplay(hg_options={'isDarkTheme': True}, viewconf={'editable': True, 'views': [{'uid': 'VtusHGBFTYmO1v…

---

## Other cool NYC properties worth looking at and comparing

In [ ]:
import pandas as pd
from utils import download_file

nyc_data = pd.read_csv('data/nyc-taxi-extended.csv')

for data in nyc_data.iterrows():
    download_file(*data[1])

In [ ]:
from higlass.tilesets import cooler

dropoff_more = {
    'passenger_count': cooler('data/nyctaxi-dropoff.passenger_count.mean.1m.mcool', name='Dropoffs Mean Passanger Count'),
    'tip_amount': cooler('data/nyctaxi-dropoff.tip_amount.mean.1m.mcool', name='Dropoffs Mean Tip'),
    'total_amount': cooler('data/nyctaxi-dropoff.total_amount.mean.1m.mcool', name='Dropoffs Mean Total Amount'),
    'trip_distance': cooler('data/nyctaxi-dropoff.trip_distance.mean.1m.mcool', name='Dropoffs Mean Trip Distance'),
}
pickup_more = {
    'passenger_count': cooler('data/nyctaxi-pickup.passenger_count.mean.1m.mcool', name='Pickups Mean Passanger Count'),
    'tip_amount': cooler('data/nyctaxi-pickup.tip_amount.mean.1m.mcool', name='Pickups Mean Tip'),
    'total_amount': cooler('data/nyctaxi-pickup.total_amount.mean.1m.mcool', name='Pickups Mean Total Amount'),
    'trip_distance': cooler('data/nyctaxi-pickup.trip_distance.mean.1m.mcool', name='Pickups Mean Trip Distance'),
}

In [ ]:
## Important change ##########
track_config = {
    'track_type': 'heatmap',
    'height': 400,
    'options': {
        'showMousePosition': True,
        'mousePositionColor': 'black',
        'showTooltip': True,
        'heatmapValueScale': 'linear',
        'valueScaleMin': 0,
    }
}
##############################

pickup_amount_track = Track(tileset=pickup_more['total_amount'], **track_config).change_options(valueScaleMax=100)
pickup_dist_track = Track(tileset=pickup_more['trip_distance'], **track_config).change_options(valueScaleMax=20)
dropoff_amount_track = Track(tileset=dropoff_more['total_amount'], **track_config).change_options(valueScaleMax=100)
dropoff_dist_track = Track(tileset=dropoff_more['trip_distance'], **track_config).change_options(valueScaleMax=20)

pickup_amount_view = View(x=0, y=0, tracks=[pickup_amount_track], **view_config)
pickup_dist_view = View(x=0, y=6, tracks=[pickup_dist_track], **view_config)
dropoff_amount_view = View(x=6, y=0, tracks=[dropoff_amount_track], **view_config)
dropoff_dist_view = View(x=6, y=6, tracks=[dropoff_dist_track], **view_config)

widget, server, viewconf = display(
    views=[
        pickup_amount_view, pickup_dist_view,
        dropoff_amount_view, dropoff_dist_view
    ],
    location_syncs=[(
        pickup_amount_view, pickup_dist_view,
        dropoff_amount_view, dropoff_dist_view
    )],
    zoom_syncs=[(
        pickup_amount_view, pickup_dist_view,
        dropoff_amount_view, dropoff_dist_view
    )],
)

widget

In [ ]:
from higlass import ViewConf
from higlass.viewer import HiGlassDisplay

## Important change ##########
pickup_div_track = (
    pickup_amount_track
        .change_options(
            name='Pickups: Amount by Distance')
        .change_attributes(
            server=server.api_address,
            # With this magic setup we tell HiGlass to divide
            # total amount by the trip distance
            data={
                'type': 'divided',
                'children': [{
                    'server': server.api_address,
                    'tilesetUid': pickup_more['total_amount'].uuid
                }, {
                    'server': server.api_address,
                    'tilesetUid': pickup_more['trip_distance'].uuid
                }]
            }
        )
)
##############################

dropoff_div_track = (
    dropoff_amount_track
        .change_options(
            name='Dropoffs: Amount by Distance')
        .change_attributes(
            server=server.api_address,
            # With this magic setup we tell HiGlass to divide
            # total amount by the trip distance
            data={
                'type': 'divided',
                'children': [{
                    'server': server.api_address,
                    'tilesetUid': dropoff_more['total_amount'].uuid
                }, {
                    'server': server.api_address,
                    'tilesetUid': dropoff_more['trip_distance'].uuid
                }]
            }
        )
)


pickup_amount_by_dist_view = View(
    x=0, y=0, tracks=[pickup_div_track], **view_config
)

dropoff_amount_by_dist_view = View(
    x=6, y=0, tracks=[dropoff_div_track], **view_config
)

## Important change ##########
div_viewconf = ViewConf([
    pickup_amount_by_dist_view,
    dropoff_amount_by_dist_view
],
location_syncs=[(
    pickup_amount_by_dist_view,
    dropoff_amount_by_dist_view
)],
zoom_syncs=[(
    pickup_amount_by_dist_view,
    dropoff_amount_by_dist_view
)])


HiGlassDisplay(viewconf=div_viewconf.to_dict())
##############################